<a href="https://colab.research.google.com/github/alexchen1999/covid-19-sample-strain-classification/blob/main/random_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Bio
!pip install bcbio-gff
!pip install ncbi-acc-download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 4.9 MB/s 
     |████████████████████████████████| 2.3 MB 35.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.6 MB/s 
  Created wheel for bcbio-gff: filename=bcbio_gff-0.6.9-py3-none-any.whl size=15954 sha256=e4b282968d396ffd7f14555db938261fab75ab15f405c14c5a951b6863f5425f
  Stored in directory: /root/.cache/pip/wheels/fb/fc/e0/2e7658046d32b794fdfd1e0ec266dc8bae21c1811b18951b6d
Successfully built bcbio-gff
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import subprocess
from Bio import Align, SeqIO
from sklearn.model_selection import train_test_split

In [3]:
covid_strains = ['B.1.1.7_sequences.csv', 'P.1_sequences.csv', 'B.1.617.2_sequences.csv', 'BA.1.1_sequences.csv']

In [5]:
alpha = pd.read_csv(covid_strains[0])
gamma = pd.read_csv(covid_strains[1])
delta = pd.read_csv(covid_strains[2])
omicron = pd.read_csv(covid_strains[3])

In [6]:
shortest = min([alpha.shape[0], gamma.shape[0], delta.shape[0], omicron.shape[0]])
print(shortest)

3735


In [7]:
alpha_sample = alpha.sample(n=shortest)
print(alpha_sample.shape[0])
gamma_sample = gamma.sample(n=shortest)
print(gamma_sample.shape[0])
delta_sample = delta.sample(n=shortest)
print(delta_sample.shape[0])
omicron_sample = omicron.sample(n=shortest)
print(omicron_sample.shape[0])

3735
3735
3735
3735


In [8]:
print(alpha_sample['Country'].unique())
print(gamma_sample['Country'].unique())
print(delta_sample['Country'].unique())
print(omicron_sample['Country'].unique())

['USA' 'Japan' 'Nigeria' 'India' 'Djibouti' 'Pakistan' 'Austria' 'Egypt'
 'West Bank' 'Spain' 'Togo' 'Finland' 'Bangladesh' 'Iraq' 'Philippines'
 'Denmark']
['USA' 'Mexico' 'Paraguay' 'Brazil' 'Taiwan' 'Chile' 'Italy'
 'Dominican Republic' 'Peru']
['USA' 'Bangladesh' 'Egypt' 'Bahrain' 'Myanmar' 'India' 'Mongolia' 'Japan'
 'Uzbekistan' 'Gabon' 'Denmark' 'Pakistan' 'Jamaica' 'Russia' 'West Bank'
 'Chile' 'China']
['USA' 'France' 'Bahrain' 'Japan']


In [9]:
# From SARS-Cov-2 GFF file, find genomic location of Spike Glycoprotein
from BCBio import GFF
in_file = "GCF_009858895.2_ASM985889v3_genomic.gff"
in_handle = open(in_file)
features = []
for rec in GFF.parse(in_handle):
    features = rec.features
in_handle.close()

start = -1
end = -1

# spike glycoprotein locus tag GU280_gp02
for i in range(len(features)):
    if "GU280_gp02" in features[i].id:
      print(features[i])
      print(features[i].location)
      print(int(features[i].location._start))
      print(int(features[i].location._end))
      start = int(features[i].location._start)
      end = int(features[i].location._end)

type: gene
location: [21562:25384](+)
id: gene-GU280_gp02
qualifiers:
    Key: Dbxref, Value: ['GeneID:43740568']
    Key: ID, Value: ['gene-GU280_gp02']
    Key: Name, Value: ['S']
    Key: gbkey, Value: ['Gene']
    Key: gene, Value: ['S']
    Key: gene_biotype, Value: ['protein_coding']
    Key: gene_synonym, Value: ['spike glycoprotein']
    Key: locus_tag, Value: ['GU280_gp02']
    Key: source, Value: ['RefSeq']

[21562:25384](+)
21562
25384


In [10]:
# Find spike glycoprotein sequence from reference genome
record = SeqIO.read("NC_045512.2.fa", "fasta")
print(record.__dict__)
print(record._seq)
print(type(record._seq))
print(str(record._seq))

# Take from 21562 to 25384
spike_seq = str(record._seq)[start:end]
print(spike_seq)
print(len(spike_seq))

{'_seq': Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA'), 'id': 'NC_045512.2', 'name': 'NC_045512.2', 'description': 'NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome', 'dbxrefs': [], 'annotations': {}, '_per_letter_annotations': {}, 'features': []}
...
<class 'Bio.Seq.Seq'>
...
3822


In [ ]:
alpha_sample_accessions = []
for i in range(shortest):
  alpha_sample_accessions.append(alpha_sample.iloc[i]['Accession'])

In [ ]:
!ncbi-acc-download --help

usage: ncbi-acc-download [-h] [-m {nucleotide,protein}] [--api-key API_KEY]
                         [-e {none,loads,all,correct}]
                         [-F {fasta,genbank,featuretable,gff3}] [-o OUT]
                         [-p PREFIX] [-g RANGE] [-r] [--url] [-v]
                         NCBI-accession [NCBI-accession ...]

positional arguments:
  NCBI-accession

optional arguments:
  -h, --help            show this help message and exit
  -m {nucleotide,protein}, --molecule {nucleotide,protein}
                        Molecule type to download. Default: nucleotide
  --api-key API_KEY     Specify USER NCBI API key. More info at
                        https://www.ncbi.nlm.nih.gov/books/NBK25497/
  -e {none,loads,all,correct}, --extended-validation {none,loads,all,correct}
                        Perform extended validation. Possible options are
                        'none' to skip validation, 'loads' to check if the
                        sequence file loads in Biopython, or '

In [ ]:
# Test NCBI accession download
test = alpha_sample_accessions[0]
print(test)

OK511623.1


In [ ]:
!ncbi-acc-download --format fasta $test

In [ ]:
for i in range(3):
  subprocess.run(["ncbi-acc-download", "--format", "fasta", alpha_sample_accessions[i]])

In [ ]:
# SeqIO
record = SeqIO.read("MZ039190.1.fa", "fasta")
print(record)  # first record
print(len(record.seq))
seq = str(record._seq)[start:end]
print(seq)
print(len(seq))

ID: MZ039190.1
Name: MZ039190.1
Description: MZ039190.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/SD-CDC-LC0043523/2021, complete genome
Number of features: 0
Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...GAA')
29694
ACTCAATTACCCCCTGCATACACTAATTCTTTCACACGTGGTGTTTATTACCCTGACAAAGTTTTCAGATCCTCAGTTTTACATTCAACTCAGGACTTGTTCTTACCTTTCTTTTCCAATGTTACTTGGTTCCATGCTATCTCTGGGACCAATGGTACTAAGAGGTTTGATAACCCTGTCCTACCATTTAATGATGGTGTTTATTTTGCTTCCACTGAGAAGTCTAACATAATAAGAGGCTGGATTTTTGGTACTACTTTAGATTCGAAGACCCAGTCCCTACTTATTGTTAATAACGCTACTAATGTTGTTATTAAAGTCTGTGAATTTCAATTTTGTAATGATCCATTTTTGGGTGTTTACCACAAAAACAACAAAAGTTGGATGGAAAGTGAGTTCAGAGTTTATTCTAGTGCGAATAATTGCACTTTTGAATATGTCTCTCAGCCTTTTCTTATGGACCTTGAAGGAAAACAGGGTAATTTCAAAAATCTTAGGGAATTTGTGTTTAAGAATATTGATGGTTATTTTAAAATATATTCTAAGCACACGCCTATTAATTTAGTGCGTGATCTCCCTCAGGGTTTTTCGGCTTTAGAACCATTGGTAGATTTGCCAATAGGTATTAACATCACTAGGTTTCAAACTTTACTTGCTTTACATAGAAGTTATTTGACTCCTGGTGATTCTTCTTCAGGTTGGACAGCTGGTGCTGCAGCTTATTATGTGGGT

In [ ]:
# Pairwise alignment
aligner = Align.PairwiseAligner()
aligner.mode = 'global'
alignments = aligner.align(seq, spike_seq)
print(alignments[0])
print(type(alignments[0]))
print(alignments[0].aligned)

A-----------CT----------C-A--A-T-T--A--C-----------C---C--CC-----T------------GCATACACTAATTCTTTCACACGTGGTGTTTATTACCCTGACAAAGTTTTCAGATCCTCAGTTTTACATTCAACTCAGGACTTGTTCTTACCTTTCTTTTCCAATGTTACTTGGTTCCATGCTAT-C-T--CT--GGGACCAATGGTACTAAGAGGTTTGATAACCCTGTCCTACCATTTAATGATGGTGTTTATTTTGCTTCCACTGAGAAGTCTAACATAATAAGAGGCTGGATTTTTGGTACTACTTTAGATTCGAAGACCCAGTCCCTACTTATTGTTAATAACGCTACTAATGTTGTTATTAAAGTCTGTGAATTTCAATTTTGTAATGATCCATTTTTGGGTGTTTA---CCACAAAAACAACAAAAGTTGGATGGAAAGTGAGTTCAGAGTTTATTCTAGTGCGAATAATTGCACTTTTGAATATGTCTCTCAGCCTTTTCTTATGGACCTTGAAGGAAAACAGGGTAATTTCAAAAATCTTAGGGAATTTGTGTTTAAGAATATTGATGGTTATTTTAAAATATATTCTAAGCACACGCCTATTAATTTAGTGCGTGATCTCCCTCAGGGTTTTTCGGCTTTAGAACCATTGGTAGATTTGCCAATAGGTATTAACATCACTAGGTTTCAAACTTTACTTGCTTTACATAGAAGTTATTTGACTCCTGGTGATTCTTCTTCAGGTTGGACAGCTGGTGCTGCAGCTTATTATGTGGGTTATCTTCAACCTAGGACTTTTCTATTAAAATATAATGAAAATGGAACCATTACAGATGCTGTAGACTGTGCACTTGACCCTCTCTCAGAAACAAAGTGTACGTTGAAATCCTTCACTGTAGAAAAAGGAATCTATCAAACTTCTAACTTTAGAGTCCAACCAACAGAATCTATTGTTAGATTTCCTAATATTACAA

In [11]:
# Credit to https://2-bitbio.com/2018/06/one-hot-encode-dna-sequence-using.html

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
alpha_spike_seq_one_hot = pd.DataFrame(columns=["Accession", "Seq"])

In [ ]:
# Alpha sample accessions
# Take from 21562 to 25384 from samples, should be a good enough approximation
alpha_spike_seq_df = pd.DataFrame(columns = ['Accession', 'Seq'])
for i in range(3):
  accession = alpha_sample_accessions[i]
  print(accession)
  subprocess.run(["ncbi-acc-download", "--format", "fasta", accession])
  record = SeqIO.read(accession + ".fa", "fasta")
  spike_seq = str(record._seq)[start:end]

  # One-hot encoding
  seq_array = array(list(spike_seq))
  label_encoder = LabelEncoder()
  integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
  #one hot the sequence
  onehot_encoder = OneHotEncoder(sparse=False)
  #reshape because that's what OneHotEncoder likes
  integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
  onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

  print(onehot_encoded_seq)

  alpha_spike_seq_df.loc[len(alpha_spike_seq_df.index)] = [accession, onehot_encoded_seq]
  subprocess.run(['rm', accession + ".fa"])

MZ039190.1
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]
MZ056250.1
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]
MW912322.1
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


In [12]:
# All
dfs = [alpha_sample, delta_sample, gamma_sample, omicron_sample]
labels = ['Alpha', 'Delta', 'Gamma', 'Omicron']
spike_seq_df = pd.DataFrame(columns = ['Accession', 'Seq', 'Label'])
for i in range(100):
  for j in range(4):
    accession = dfs[j].iloc[i]['Accession']

    subprocess.run(["ncbi-acc-download", "--format", "fasta", accession])
    record = SeqIO.read(accession + ".fa", "fasta")
    spike_seq = str(record._seq)[start:end]

    # One-hot encoding
    seq_array = array(list(spike_seq))
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
    #one hot the sequence
    onehot_encoder = OneHotEncoder(sparse=False)
    #reshape because that's what OneHotEncoder likes
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

    spike_seq_df.loc[len(spike_seq_df.index)] = [accession, onehot_encoded_seq, labels[j]]
    subprocess.run(['rm', accession + ".fa"])
  print(i+1, "% complete")

df = shuffle(spike_seq_df)
df.to_csv('spike_seq_one_hots.csv')

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
1 % complete
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]
2 % complete
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [

In [14]:
# Final check that df is well-formed
df
print(len(df.iloc[0]['Seq']))

3822
